## 101 - Training and Evaluating Classifiers with `mmlspark`

In this example, we try to predict incomes from the *Adult Census* dataset.

First, we import the packages (use `help(mmlspark)` to view contents),

In [0]:
import numpy as np
import pandas as pd

In [0]:
import mmlspark

Now let's read the data and split it to train and test sets:

In [0]:
data = spark.read.parquet("wasbs://publicwasb@mmlspark.blob.core.windows.net/AdultCensusIncome.parquet")
data = data.select(["education", "marital-status", "hours-per-week", "income"])
train, test = data.randomSplit([0.75, 0.25], seed=123)
train.limit(10).toPandas()

,education,marital-status,hours-per-week,income
0,10th,Divorced,1.0,<=50K
1,10th,Divorced,40.0,<=50K
2,10th,Divorced,40.0,<=50K
3,10th,Divorced,40.0,<=50K
4,10th,Divorced,40.0,<=50K
5,10th,Divorced,40.0,<=50K
6,10th,Divorced,40.0,<=50K
7,10th,Divorced,40.0,<=50K
8,10th,Divorced,68.0,>50K
9,10th,Married-civ-spouse,16.0,<=50K


`TrainClassifier` can be used to initialize and fit a model, it wraps SparkML classifiers.
You can use `help(mmlspark.TrainClassifier)` to view the different parameters.

Note that it implicitly converts the data into the format expected by the algorithm: tokenize
and hash strings, one-hot encodes categorical variables, assembles the features into a vector
and so on.  The parameter `numFeatures` controls the number of hashed features.

In [0]:
from mmlspark.train import TrainClassifier
from pyspark.ml.classification import LogisticRegression
model = TrainClassifier(model=LogisticRegression(), labelCol="income", numFeatures=256).fit(train)

In [0]:
help(TrainClassifier)

Help on class TrainClassifier in module mmlspark.train.TrainClassifier:

class TrainClassifier(mmlspark.train._TrainClassifier._TrainClassifier)
 Args:
 
 featuresCol (str): The name of the features column (default: [self.uid]_features)
 labelCol (str): The name of the label column
 labels (list): Sorted label values on the labels column
 model (object): Classifier to run
 numFeatures (int): Number of features to hash to (default: 0)
 reindexLabel (bool): Re-index the label column (default: true)
 
 Method resolution order:
 TrainClassifier
 mmlspark.train._TrainClassifier._TrainClassifier
 mmlspark.core.schema.Utils.ComplexParamsMixin
 pyspark.ml.util.JavaMLReadable
 pyspark.ml.util.MLReadable
 pyspark.ml.util.JavaMLWritable
 pyspark.ml.util.MLWritable
 pyspark.ml.wrapper.JavaEstimator
 pyspark.ml.wrapper.JavaParams
 pyspark.ml.wrapper.JavaWrapper
 pyspark.ml.base.Estimator
 pyspark.ml.param.Params
 pyspark.ml.util.Identifiable
 builtins.object
 
 Methods inherited from mmlspark.train._TrainClassifier._TrainClassifier:
 
 __init__(self, featuresCol=None, labelCol=None, labels=None, model=None, numFeatures=0, reindexLabel=True)
 Initialize self. See help(type(self)) for accurate signature.
 
 getFeaturesCol(self)
 Returns:
 
 str: The name of the features column (default: [self.uid]_features)
 
 getLabelCol(self)
 Returns:
 
 str: The name of the label column
 
 getLabels(self)
 Returns:
 
 list: Sorted label values on the labels column
 
 getModel(self)
 Returns:
 
 object: Classifier to run
 
 getNumFeatures(self)
 Returns:
 
 int: Number of features to hash to (default: 0)
 
 getReindexLabel(self)
 Returns:
 
 bool: Re-index the label column (default: true)
 
 setFeaturesCol(self, value)
 Args:
 
 featuresCol: The name of the features column (default: [self.uid]_features)
 
 setLabelCol(self, value)
 Args:
 
 labelCol: The name of the label column
 
 setLabels(self, value)
 Args:
 
 labels: Sorted label values on the labels column
 
 setModel(self, value)
 Args:
 
 model: Classifier to run
 
 setNumFeatures(self, value)
 Args:
 
 numFeatures: Number of features to hash to (default: 0)
 
 setParams(self, featuresCol=None, labelCol=None, labels=None, model=None, numFeatures=0, reindexLabel=True)
 Set the (keyword only) parameters
 
 Args:
 
 featuresCol (str): The name of the features column (default: [self.uid]_features)
 labelCol (str): The name of the label column
 labels (list): Sorted label values on the labels column
 model (object): Classifier to run
 numFeatures (int): Number of features to hash to (default: 0)
 reindexLabel (bool): Re-index the label column (default: true)
 
 setReindexLabel(self, value)
 Args:
 
 reindexLabel: Re-index the label column (default: true)
 
 ----------------------------------------------------------------------
 Class methods inherited from mmlspark.train._TrainClassifier._TrainClassifier:
 
 read() from builtins.type
 Returns an MLReader instance for this class.
 
 ----------------------------------------------------------------------
 Static methods inherited from mmlspark.train._TrainClassifier._TrainClassifier:
 
 getJavaPackage()
 Returns package name String.
 
 ----------------------------------------------------------------------
 Class methods inherited from pyspark.ml.util.MLReadable:
 
 load(path) from builtins.type
 Reads an ML instance from the input path, a shortcut of `read().load(path)`.
 
 ----------------------------------------------------------------------
 Data descriptors inherited from pyspark.ml.util.MLReadable:
 
 __dict__
 dictionary for instance variables (if defined)
 
 __weakref__
 list of weak references to the object (if defined)
 
 ----------------------------------------------------------------------
 Methods inherited from pyspark.ml.util.JavaMLWritable:
 
 write(self)
 Returns an MLWriter instance for this ML instance.
 
 ----------------------------------------------------------------------
 Methods inherited from pyspark.ml.util.MLWritable:
 
 save(se

After the model is trained, we score it against the test dataset and view metrics.

In [0]:
from mmlspark.train import ComputeModelStatistics, TrainedClassifierModel
prediction = model.transform(test)
metrics = ComputeModelStatistics().transform(prediction)
metrics.limit(10).toPandas()

,evaluation_type,confusion_matrix,accuracy,precision,recall,AUC
0,Classification,"DenseMatrix([[5780., 378.],\n [10...",0.825283,0.708333,0.468846,0.87245


Finally, we save the model so it can be used in a scoring program.

In [0]:
model.write().overwrite().save("AdultCensus.mml")